In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

# Adjust these paths as needed (for local files or from Google Drive)
model_id = "bigscience/bloom-7b1"
DATA_DIR = "/content/drive/MyDrive/NLP_project/cleaned_files"

# Load model and tokenizer with quantization (8‑bit) to fit in a Colab GPU
print("\nLoading large LLM...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model loaded on: {device}")



Loading large LLM...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on: cuda


In [ ]:
import torch

def get_model_size_mb(model):
    # Calculate the total size of all parameters in bytes.
    param_size = sum(p.nelement() * p.element_size() for p in model.parameters())
    buffer_size = sum(b.nelement() * b.element_size() for b in model.buffers())
    total_size_mb = (param_size + buffer_size) / (1024 ** 2)
    return total_size_mb

def print_gpu_usage():
    allocated = torch.cuda.memory_allocated()  # Currently allocated memory in bytes
    reserved = torch.cuda.memory_reserved()    # Currently reserved memory in bytes
    max_allocated = torch.cuda.max_memory_allocated()  # Peak allocated memory
    max_reserved = torch.cuda.max_memory_reserved()    # Peak reserved memory
    print(f"GPU Memory Allocated: {allocated / (1024 ** 3):.2f} GB")
    print(f"GPU Memory Reserved: {reserved / (1024 ** 3):.2f} GB")
    print(f"GPU Max Memory Allocated: {max_allocated / (1024 ** 3):.2f} GB")
    print(f"GPU Max Memory Reserved: {max_reserved / (1024 ** 3):.2f} GB")

# Assuming 'model' is your loaded PyTorch model and it's on the GPU
model_size_mb = get_model_size_mb(model)
print(f"Model size: {model_size_mb:.2f} MB")

# Print current GPU memory usage
print_gpu_usage()


Model size: 26966.16 MB
GPU Memory Allocated: 26.33 GB
GPU Memory Reserved: 26.34 GB
GPU Max Memory Allocated: 26.33 GB
GPU Max Memory Reserved: 26.34 GB


In [ ]:
def load_corpus(data_dir, start_index=7000, max_files=100):
    texts = []
    summaries = []

    filenames = sorted(os.listdir(data_dir))
    for idx, filename in enumerate(filenames):
        print(f"Loading file {idx + 1}/{len(filenames)}")
        if idx < start_index:
            continue
        if idx >= start_index + max_files:
            break
        if filename.endswith(".story"):
            with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
                content = f.read()
                if "@highlight" in content:
                    text, summary = content.split("@highlight", 1)
                    texts.append(text.strip())
                    summaries.append(summary.strip())
    return Dataset.from_dict({"text": texts, "summary": summaries})

print("\nLoading data...")
train_dataset = load_corpus(DATA_DIR, start_index=5000, max_files=1000)
print(f"Number of files loaded: {len(train_dataset)}")


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Loading file 1003/12001
Loading file 1004/12001
Loading file 1005/12001
Loading file 1006/12001
Loading file 1007/12001
Loading file 1008/12001
Loading file 1009/12001
Loading file 1010/12001
Loading file 1011/12001
Loading file 1012/12001
Loading file 1013/12001
Loading file 1014/12001
Loading file 1015/12001
Loading file 1016/12001
Loading file 1017/12001
Loading file 1018/12001
Loading file 1019/12001
Loading file 1020/12001
Loading file 1021/12001
Loading file 1022/12001
Loading file 1023/12001
Loading file 1024/12001
Loading file 1025/12001
Loading file 1026/12001
Loading file 1027/12001
Loading file 1028/12001
Loading file 1029/12001
Loading file 1030/12001
Loading file 1031/12001
Loading file 1032/12001
Loading file 1033/12001
Loading file 1034/12001
Loading file 1035/12001
Loading file 1036/12001
Loading file 1037/12001
Loading file 1038/12001
Loading file 1039/12001
Loading file 1040/12001
Loading fi

In [ ]:
max_length = tokenizer.model_max_length
print(f"Model max length: {max_length}")

Model max length: 1000000000000000019884624838656


In [ ]:
def generate_summary(text, max_total_tokens=2048):
    # Définir le préfixe d'instructions
    prefix = (
        "Vous êtes un expert en synthèse de texte. Veuillez fournir un résumé détaillé et complet du texte suivant.\n"
        "Assurez-vous d'inclure tous les détails importants et l'essence générale du contenu en passant des lignes pour la lisibilité.\n"
        "Texte : "
    )

    # Encoder le préfixe pour obtenir sa longueur (sans tokens spéciaux)
    prefix_ids = tokenizer.encode(prefix, add_special_tokens=False)

    # Calculer le nombre de tokens autorisés pour le texte à résumer
    allowed_text_tokens = max_total_tokens - len(prefix_ids)

    # Encoder le texte à résumer avec troncature si nécessaire
    text_ids = tokenizer.encode(
        text,
        add_special_tokens=False,
        truncation=True,
        max_length=allowed_text_tokens
    )

    # Combiner le préfixe et le texte tronqué, puis ajouter le marqueur "Résumé:"
    combined_ids = prefix_ids + text_ids
    prompt = tokenizer.decode(combined_ids)
    prompt += "\n\nRésumé:"  # Ajout explicite du marqueur

    # Préparer l'entrée pour le modèle
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Conserver la liste des tokens du prompt
    prompt_token_list = inputs.input_ids[0].tolist()

    # Générer le résumé avec beam search et autres paramètres ajustables
    summary_ids = model.generate(
        inputs.input_ids,
        max_new_tokens=512,  # Ajustez selon la longueur souhaitée pour le résumé
        num_beams=2,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id
    )

    # Convertir la sortie générée en liste de tokens
    output_token_list = summary_ids[0].tolist()

    # Si le début de la sortie correspond exactement aux tokens du prompt, supprimer cette partie
    if output_token_list[:len(prompt_token_list)] == prompt_token_list:
        generated_token_list = output_token_list[len(prompt_token_list):]
    else:
        generated_token_list = output_token_list

    # Décoder les tokens générés pour obtenir le résumé
    summary = tokenizer.decode(
        generated_token_list,
        skip_special_tokens=True
    ).strip()

    # Optionnel : si le marqueur "Résumé:" apparaît toujours dans le texte, extraire uniquement ce qui suit
    if "Résumé:" in summary:
        summary = summary.split("Résumé:", 1)[1].strip()

    return summary


In [ ]:
torch.cuda.empty_cache()
model = model.half()
model.eval()  # Ensure the model is in evaluation mode

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 4096)
    (word_embeddings_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
          (dense): Linear(in_features=4096, out_features=4096, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=4096, out_features=16384, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )


In [ ]:
import pandas as pd
from datasets import Dataset
from tqdm.notebook import tqdm  # For a nicer display in Colab
import os

# Define your save directory (make sure it exists)
save_dir = "/content/drive/MyDrive/NLP_project"
os.makedirs(save_dir, exist_ok=True)

generated_summaries = []
original_texts = []
reference_summaries = []  # if available

batch_size = 1000  # Save every 1000 texts
batch_counter = 4

print("\nGenerating summaries for all documents...")
for idx, example in enumerate(tqdm(train_dataset, desc="Generating summaries", total=len(train_dataset))):
    text = example["text"]
    # Generate the summary using your defined function
    summary = generate_summary(text)
    generated_summaries.append(summary)
    original_texts.append(text)
    # Optionally include the reference summary if available
    reference_summaries.append(example.get("summary", ""))

    # When we've processed a batch, save it
    if (idx + 1) % batch_size == 0:
        batch_counter += 1
        output_df = pd.DataFrame({
            "text": original_texts,
            "generated_summary": generated_summaries,
            "reference_summary": reference_summaries
        })

        # Save to CSV
        csv_filename = os.path.join(save_dir, f"generated_summaries_batch_{batch_counter}.csv")
        output_df.to_csv(csv_filename, index=False)
        # Save to pickle
        pkl_filename = os.path.join(save_dir, f"generated_summaries_batch_{batch_counter}.pkl")
        output_df.to_pickle(pkl_filename)

        print(f"Saved batch {batch_counter} with {batch_size} texts to {csv_filename} and {pkl_filename}.")

        # Clear lists for next batch
        generated_summaries = []
        original_texts = []
        reference_summaries = []

# Save any remaining texts that didn't form a complete batch
if generated_summaries:
    batch_counter += 1
    output_df = pd.DataFrame({
        "text": original_texts,
        "generated_summary": generated_summaries,
        "reference_summary": reference_summaries
    })
    csv_filename = os.path.join(save_dir, f"generated_summaries_batch_{batch_counter}.csv")
    output_df.to_csv(csv_filename, index=False)
    pkl_filename = os.path.join(save_dir, f"generated_summaries_batch_{batch_counter}.pkl")
    output_df.to_pickle(pkl_filename)
    print(f"Saved final batch {batch_counter} with {len(generated_summaries)} texts to {csv_filename} and {pkl_filename}.")


# # --- Later on, you can reload the data and convert it to a Hugging Face Dataset ---
# # For example, loading from the pickle file:
# df_loaded = pd.read_pickle("all_generated_summaries.pkl")
# hf_dataset = Dataset.from_pandas(df_loaded)
# print("Dataset loaded from Pandas DataFrame:")
# print(hf_dataset)



Generating summaries for all documents...


Generating summaries:   0%|          | 0/1000 [00:00<?, ?it/s]

Saved batch 5 with 1000 texts to /content/drive/MyDrive/NLP_project/generated_summaries_batch_5.csv and /content/drive/MyDrive/NLP_project/generated_summaries_batch_5.pkl.
